In [ ]:
import utility 
import numpy as np
from importlib import reload
reload(utility)
import sys, os, shutil
import time
from tqdm import tqdm
import matplotlib.pyplot as pl
from annoy import AnnoyIndex

HOME = '/cluster/home/ajoudaki/projects2019-string-embedding/python_tools'

file_name = 'seqs9'
data_path = HOME + '/data/' + file_name + '.npz'
result_path = HOME + '/results/' + file_name 
index_path = result_path + '/index/'
search_path = result_path + '/search/'

if not os.path.exists(result_path):
    os.makedirs(result_path)
if not os.path.exists(index_path):
    os.makedirs(index_path)
if not os.path.exists(search_path):
    os.makedirs(search_path)


Res = np.load(data_path)


seqs = Res['seqs']
vals = Res['vals']
options = Res['options']
Op = (options[()])
if int(file_name[4:])>=7:
    gene_lens = Res['gene_lens']
    num_seqs = Res['num_seqs']
else:
    num_seqs = [Op.num_seq]*Op.repeat

print('num seqs = ', len(seqs), ' mean lenth of seqs = ', np.mean([len(seqs[i]) for i in range(len(seqs))]))
print(Op)

num seqs =  15  mean lenth of seqs =  19030009.2667
{'mutation_rate': 0.1, 'gene_len': 100, 'num_genes': 1000, 'padding': 9000, 'num_seq': 5, 'repeat': 3, 'geometric_p': 0.4, 'reverse_p': 0, 'gene_len2': 2000, 'num_seq2': 5, 'colored': False, 'print_values': False, 'print_seqs': False, 'save_directory': '../data/seqs9'}


In [ ]:
proj_dim = 120
k_small = 4 
k_big = 100
num_trees = 2
num_neighbours = 200
step_build = 5
step_search = 5
metric = 'euclidean'


kmers, s_kmer_vals, kmer_pos, kmer_seq_id = utility.list_kmers_simple(seqs, vals = vals,  
                                         k = k_small, addy = True, padding = int(k_big))
kmer_vals = utility.get_kmver_vals(s_kmer_vals, k_big)
build_indices = np.arange(0,len(kmers),step_build)
search_indices = np.arange(0,len(kmers),step_search)

convolved = utility.random_projection(kmers, k_big, proj_dim)

knn_index = utility.build_index(matrix = convolved, 
                                indices = build_indices, 
                                num_trees = num_trees, 
                                metric = metric, 
                                index_path = index_path + '0.ann')

knn_index.save(index_path + '0.ann')
index = AnnoyIndex(proj_dim, metric= metric) 
index.load(index_path + '0.ann')
NN,NN_dist = utility.knn_search_value(knn_index, convolved, search_indices, num_neighbours)
    
quad = utility.eval_results(search_indices, NN, kmers, kmer_vals, kmer_pos, kmer_seq_id, num_seqs, options)


print('hit ratio: ', len(quad)/len(search_indices)/num_neighbours)

computing the list of kmers and values 


100%|██████████| 15/15 [03:33<00:00, 14.21s/it]


 computing the projection


100%|██████████| 285453094/285453094 [06:23<00:00, 745293.19it/s]

 computing the convolution



  0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
# print(Op)